In [24]:
%pwd

'f:\\Machine_learning_end_to_end\\machine_learning\\notebooks'

In [25]:
import os

In [26]:
os.chdir("../")

In [27]:
%pwd

'f:\\Machine_learning_end_to_end\\machine_learning'

In [28]:
from dataclasses import dataclass
from src.constant import *

In [29]:
@dataclass
class ModelTrainerConfig:
    pkl_obj_path:Path
    model_path:Path
    alpha:float
    l1_ratio:float
    TARGET_COLUMN:str
    metric_file_name:Path

In [30]:
from src.utils.common import read_yaml,save_json


class Configuration:
    def __init__(self,config_file_path=CONFIG_FILE_PATH,
                 schema_file_path=SCHEMA_FILE_PATH,
                 param_file_path=PARAMS_FILE_PATH):
        self.config=read_yaml(config_file_path)
        self.schema=read_yaml(schema_file_path)
        self.params=read_yaml(param_file_path)

    def model_trainer_config(self):
        config=self.config.model_trainer
        schema=self.schema.TARGET_COLUMN
        params=self.params.ElasticNet

        model_trainer_config=ModelTrainerConfig(
            pkl_obj_path=config.pkl_obj_path,
            model_path=config.model_path,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            TARGET_COLUMN=schema.name,
            metric_file_name=config.metric_file_name

        )
        return model_trainer_config

In [31]:
import pandas as pd
import os
from src import logger
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error,r2_score
import joblib

In [36]:

class ModelTrainer:
    def __init__(self,config:ModelTrainerConfig):
        self.config=config

    def evaluate(self,X_train,Y_train,X_test,Y_test):
        lr=ElasticNet(alpha=self.config.alpha,l1_ratio=self.config.l1_ratio)
        lr.fit(X_train,Y_train)

        y_pred=lr.predict(X_test)
        r2_scores=r2_score(Y_test,y_pred)
        mae_score=mean_squared_error(Y_test,y_pred)
        logger.info(f"accuracy score is {r2_scores}")
        logger.info(f"mean squared error is {mae_score}")
        return r2_scores,mae_score


    def train_model(self,train_arr,test_arr):
        logger.info("Model Trainer Started")
        X_train=train_arr[:,:-1]
        logger.info(f"X train shape is {pd.DataFrame(X_train).shape}")
        Y_train=train_arr[:,-1]
        logger.info(f"Y train shape is {pd.DataFrame(Y_train).shape}")
        X_test=test_arr[:,:-1]
        logger.info(f"X test shape is {pd.DataFrame(X_test).shape}")
        Y_test=test_arr[:,-1]
        logger.info(f"Y test shape is {pd.DataFrame(Y_test).shape}")
        r2_scores,mae_score=self.evaluate(X_train,Y_train,X_test,Y_test)
        scores={"accuracy":r2_scores,"mae":mae_score}
        save_json(Path(self.config.metric_file_name),scores)

In [37]:
from src.configuration.config import ConfigurationManager
from src.components.data_transformation import DataTransformation
from  src.entity.config_entity import DataTransformationConfig

In [38]:
obj=ConfigurationManager()
data_trans=obj.data_transformation_config()
obj2=DataTransformation(data_trans)
train_arr,test_arr=obj2.train_test_split()

[2024-11-17 20:35:17,258 : INFO : common : yaml file path : config\config.yaml loaded successfully:]
[2024-11-17 20:35:17,305 : INFO : common : yaml file path : schema.yaml loaded successfully:]
[2024-11-17 20:35:17,316 : INFO : common : yaml file path : params.yaml loaded successfully:]
[2024-11-17 20:35:18,500 : INFO : data_transformation : data are splitted into train and test]
[2024-11-17 20:35:18,600 : INFO : data_transformation : train data shape (914, 12)]
[2024-11-17 20:35:18,603 : INFO : data_transformation : test data shape (229, 12)]
[2024-11-17 20:35:18,608 : INFO : data_transformation : train data column : ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol', 'quality']
]
[2024-11-17 20:35:18,612 : INFO : data_transformation : test data column ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total s

In [39]:
obj3=Configuration()
trainer_config=obj3.model_trainer_config()
obj4=ModelTrainer(trainer_config)
obj4.train_model(train_arr,test_arr)

[2024-11-17 20:35:19,887 : INFO : common : yaml file path : config\config.yaml loaded successfully:]
[2024-11-17 20:35:19,897 : INFO : common : yaml file path : schema.yaml loaded successfully:]
[2024-11-17 20:35:19,902 : INFO : common : yaml file path : params.yaml loaded successfully:]
[2024-11-17 20:35:19,907 : INFO : 2563309750 : Model Trainer Started]
[2024-11-17 20:35:19,910 : INFO : 2563309750 : X train shape is (914, 11)]
[2024-11-17 20:35:19,914 : INFO : 2563309750 : Y train shape is (914, 1)]
[2024-11-17 20:35:19,917 : INFO : 2563309750 : X test shape is (229, 11)]
[2024-11-17 20:35:19,921 : INFO : 2563309750 : Y test shape is (229, 1)]
[2024-11-17 20:35:19,976 : INFO : 2563309750 : accuracy score is 0.2902765754040312]
[2024-11-17 20:35:19,979 : INFO : 2563309750 : mean squared error is 0.4383852552299273]
[2024-11-17 20:35:19,987 : INFO : common : json file save to path : data\model_evaluation\metrics.json]
